In [1]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json

import cv2
import torch
from tqdm import tqdm_notebook
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
import torchvision

/home/briancy2/.conda/envs/iwildcam/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
META_DIR = "data/metadata/metadata/"
TRAIN_DIR = "data/train/train/"

test_data = json.load(open(META_DIR + 'iwildcam2022_test_information.json'))
train_data = json.load(open(META_DIR + 'iwildcam2022_train_annotations.json'))

#Test images
df_test = pd.DataFrame({'id': [item['id'] for item in test_data['images']],
                                'seq_id': [item['seq_id'] for item in test_data['images']],
                                'file_name': [item['file_name'] for item in test_data['images']],
                                'location': [item['location'] for item in test_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in test_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in test_data['images']]})
#Train images
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['images']],
                                'seq_id': [item['seq_id'] for item in train_data['images']],
                                'file_name': [item['file_name'] for item in train_data['images']],
                                'location': [item['location'] for item in train_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in train_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in train_data['images']]})
# Detection for train test
detections = json.load(open(META_DIR+"iwildcam2022_mdv4_detections.json"))['images']
det_dict = dict()
for detection in tqdm(detections):
    det_dict[detection['file']] = detection['detections']
df_detection = pd.DataFrame({'file': [item['file'] for item in detections],
                                'detections': [item['detections'] for item in detections]})
# Test sequence ids
test_sequence_ids = pd.unique(df_test['seq_id'])
# Train sequence id and count
train_seq_count = pd.read_csv(META_DIR+"train_sequence_counts.csv")

In [20]:
print(seq_count)

                                    seq_id  count
0     95dae922-21bc-11ea-a13a-137349068a90      9
1     2fff6c8a-7d42-11eb-8fb5-0242ac1c0002      9
2     300cc650-7d42-11eb-8fb5-0242ac1c0002      9
3     30125958-7d42-11eb-8fb5-0242ac1c0002      9
4     301d2d24-7d42-11eb-8fb5-0242ac1c0002      9
...                                    ...    ...
1775  9925c372-21bc-11ea-a13a-137349068a90      8
1776  9932df3a-21bc-11ea-a13a-137349068a90      8
1777  99556aa0-21bc-11ea-a13a-137349068a90      5
1778  99577458-21bc-11ea-a13a-137349068a90      4
1779  99592c3a-21bc-11ea-a13a-137349068a90      5

[1780 rows x 2 columns]


In [ ]:
for sequence in sequences:
   cur_idx_row = self.seq_count.iloc[idx]
#         y = int(cur_idx_row['count'])
#         img_rows = self.df.loc[self.df.seq_id == cur_idx_row['seq_id']]
#         out = torch.zeros(1024,1024,3,img_rows.iloc[0].seq_num_frames)
#         for index, img_row in img_rows.iterrows():
#            img_path = os.path.join(self.images_dir, img_row['file_name'])

In [ ]:
for sequence in sequences:
        print("Running sequence %s" % sequence)
        sequence_dir = os.path.join(args.mot_dir, sequence)
        detection_file = os.path.join(args.detection_dir, "%s.npy" % sequence)
        output_file = os.path.join(args.output_dir, "%s.txt" % sequence)
        deep_sort_app.run(
            sequence_dir, detection_file, output_file, args.min_confidence,
            args.nms_max_overlap, args.min_detection_height,
            args.max_cosine_distance, args.nn_budget, display=False)
        
        
    seq_info = gather_sequence_info(sequence_dir, detection_file)
    metric = nn_matching.NearestNeighborDistanceMetric(
        "cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)
    results = []
        detections = create_detections(
            seq_info["detections"], frame_idx, min_detection_height)
        for row in detection_mat[mask]:
        bbox, confidence, feature = row[2:6], row[6], row[10:]
        detections = [d for d in detections if d.confidence >= min_confidence]

        # Run non-maxima suppression.
        boxes = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        indices = preprocessing.non_max_suppression(
            boxes, nms_max_overlap, scores)
        detections = [detections[i] for i in indices]

        # Update tracker.
        tracker.predict()
        tracker.update(detections)

        # Update visualization.
        if display:
            image = cv2.imread(
                seq_info["image_filenames"][frame_idx], cv2.IMREAD_COLOR)
            vis.set_image(image.copy())
            vis.draw_detections(detections)
            vis.draw_trackers(tracker.tracks)

        # Store results.
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlwh()
            results.append([
                frame_idx, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])

    # Run tracker.
    if display:
        visualizer = visualization.Visualization(seq_info, update_ms=5)
    else:
        visualizer = visualization.NoVisualization(seq_info)
    visualizer.run(frame_callback)

    # Store results.
    f = open(output_file, 'w')
    for row in results:
        print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1' % (
            row[0], row[1], row[2], row[3], row[4], row[5]),file=f)


def bool_string(input_string):
    if input_string not in {"True","False"}:
        raise ValueError("Please Enter a valid Ture/False choice")
    else:
        return (input_string == "True")